# Pandas

## Learning Objectives
After completing this session you should be able to:
1. Combine datasets with **`concat`**, **`merge`**, and joins.
2. Reshape data with **`pivot_table`**, **`melt`**, **`stack`**, and **`unstack`**.
3. Work with **MultiIndex** objects for hierarchical data organization.
4. Manipulate and analyze **time‑series** data (resampling & rolling windows).
5. Use **categorical** types to improve performance & semantics.
6. Apply performance optimizations for large datasets.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.width = 100
pd.options.display.max_columns = 20

## 1 · Combining DataFrames

In [ ]:
# Example data
df_a = pd.DataFrame({'id': [1, 2, 3], 'name': ['Alice', 'Bob', 'Charlie']})
df_b = pd.DataFrame({'id': [2, 3, 4], 'dept': ['Finance', 'HR', 'Engineering']})
df_a, df_b

In [ ]:
# Inner join (intersection of keys)
pd.merge(df_a, df_b, on='id', how='inner')

In [ ]:
df_a

In [ ]:
df_b

In [ ]:
# Outer join (union of keys)
pd.merge(df_a, df_b, on='id', how='outer')

In [ ]:
# Concatenate row‑wise
df_c = pd.DataFrame({'id': [4, 5], 'name': ['Dan', 'Eva']})
pd.concat([df_a, df_c], ignore_index=True)

In [ ]:
# Concatenate column‑wise
pd.concat([df_a.set_index('id'), df_b.set_index('id')], axis=1)

## 2 · Reshaping & Pivoting

In [ ]:
# Long to wide with pivot_table
sales_long = pd.DataFrame({
    'store': ['A', 'A', 'B', 'B'] * 3,
    'quarter': ['Q1']*4 + ['Q2']*4 + ['Q3']*4,
    'product': ['widgets', 'gadgets', 'widgets', 'gadgets']*3,
    'revenue': np.random.randint(1000, 5000, 12)
})
sales_wide = sales_long.pivot_table(index=['store', 'product'], columns='quarter', values='revenue')
sales_wide

In [ ]:
# Wide to long with melt
sales_melted = sales_wide.reset_index().melt(id_vars=['store', 'product'], var_name='quarter', value_name='revenue')
sales_melted.head()

In [ ]:
# stack / unstack
sales_wide.stack().head()

## 3 · Hierarchical Indexing (MultiIndex)

In [ ]:
tuples = [('USA', 'NY'), ('USA', 'CA'), ('UK', 'London'), ('UK', 'Manchester')]
index = pd.MultiIndex.from_tuples(tuples, names=['country', 'city'])
pop = pd.Series([19.8, 39.0, 8.9, 2.8], index=index, name='population (millions)')
pop

In [ ]:
# Cross‑section (all rows for country 'USA')
pop.xs('USA')

In [ ]:
# Swap levels and sort
pop.swaplevel().sort_index().head()

## 5 · Categorical Data

In [ ]:
survey = pd.DataFrame({'response': np.random.choice(['agree', 'neutral', 'disagree'], 1000)})
survey['response_cat'] = pd.Categorical(survey['response'], categories=['disagree', 'neutral', 'agree'], ordered=True)
survey['response_cat'].value_counts().sort_index()

## 6 · Performance & Memory Efficiency

In [ ]:
# Simulate a large DataFrame
n = 1_000_000
big = pd.DataFrame({
    'int_col': np.random.randint(0, 100, n),
    'float_col': np.random.rand(n),
    'str_col': np.random.choice(['A', 'B', 'C', 'D'], n)
})
mem_before = big.memory_usage(deep=True).sum() / 1024**2
mem_before

In [ ]:
# Downcast numeric columns and convert object to category
opt = big.copy()
opt['int_col'] = pd.to_numeric(opt['int_col'], downcast='unsigned')
opt['float_col'] = pd.to_numeric(opt['float_col'], downcast='float')
opt['str_col'] = opt['str_col'].astype('category')
mem_after = opt.memory_usage(deep=True).sum() / 1024**2
mem_before, mem_after

### Vectorization vs. `apply`
Whenever possible, rely on built‑in vectorized operations instead of row‑wise `apply`, which is slower.

## 7 · Hands‑on Exercises

1. **Order analysis**  
   *Download* the Kaggle "Brazilian E‑Commerce Public Dataset by Olist" (or another sales dataset). Merge the `orders`, `order_items`, and `products` tables to compute total revenue **per product category**.

2. **Temperature heatmap**  
   Load the classic *flights* dataset from Seaborn (`sns.load_dataset('flights')`). Pivot it to shape **months × year** with passenger counts, then plot a heatmap (hint: use `pivot_table`).

3. **Rolling stock prices**  
   Fetch daily closing prices for two tickers using `pandas_datareader`. Compute and plot the 20‑day rolling standard deviation for each.

4. **Memory optimization**  
   Create a function `memory_report(df)` that prints the memory usage of every column. Apply it to a large CSV you have, then optimize dtypes and show the improvement.